# 10-armed Testbed

In [1]:
import numpy as np
from tqdm import trange
import matplotlib
import matplotlib.pyplot as plt
import os

# Import the Bandit module
from banditt import Bandit

# Use 'Agg' backend for headless environments
matplotlib.use('Agg')

In [2]:
def simulate(runs, times, bandits):
    # region Summary
    """
    For any learning method, we can measure its performance and behavior as it improves with experience over 1000 time steps 
    when applied to 1 of the bandit problems. This makes up 1 run. Repeating this for 2000 independent runs, each with a different 
    bandit problem, we obtained measures of the learning algorithm’s average behavior.
    :param runs: Number of runs
    :param times: Number of times
    :param bandits: Bandit problems
    :return: Optimal action count mean and reward mean
    """
    # endregion Summary
    
    # region Body
    
    # Prepare a matrix filled with 0s for rewards
    rewards = np.zeros((len(bandits), runs, times))

    # Prepare a matrix filled with 0s for optimal action counts that has the same shape as rewards matrix
    optimal_action_counts = np.zeros(rewards.shape)

    # For every bandit
    for i, bandit in enumerate(bandits):
        # for every run
        for run in trange(runs):
            # initialize bandit
            bandit.initialize()

            # for every time step
            for time in range(times):
                # select an action
                action = bandit.act()

                # get the reward
                rewards[i, run, time] = bandit.step(action)

                # if the selected action is optimal for bandit
                if action == bandit.optimal_action:
                    # change the corresponding 0 in the optimal action counts matrix to 1
                    optimal_action_counts[i, run, time] = 1

    return optimal_action_counts.mean(axis=1), rewards.mean(axis=1)

## 1. Reward Distribution

In [ ]:
# Generate example reward distributions
np.random.seed(42)
dataset = np.random.randn(200, 10) + np.random.randn(10)

# Ensure directory exists
os.makedirs("../generated_images", exist_ok=True)

# Plot and save
plt.violinplot(dataset, showmeans=True, showmedians=True)
plt.title("Figure 2.1")
plt.xlabel("Action")
plt.ylabel("Reward distribution")
plt.savefig("../generated_images/figure_2_1.png")
plt.close()

# Print success message
print("Plot saved successfully at: ../generated_images/figure_2_1.png")

## 2. Greedy Action Selection VS ε-greedy Action Selection

In [ ]:
# Create a list of epsilons with 0, 0.1, and 0.01 values
epsilons = [0, 0.1, 0.01]

# Create a list of bandits (1 bandit for every epsilon) where every bandit uses the sample-average method
bandits = [Bandit(epsilon=e, use_sample_averages=True) for e in epsilons]

In [ ]:
# Define number of runs
runs = 5000

# Define number of times
times = 10000

# Simulate optimal action counts and rewards
optimal_action_counts, rewards_mean = simulate(runs, times, bandits)

print("Optimal Actions Mean:", optimal_action_counts)
print("Rewards Mean:", rewards_mean)

In [ ]:
# Plotting
plt.figure(figsize=(10, 20))

In [ ]:
# Plot average rewards over time
plt.subplot(2, 1, 1)
for epsilon, rewards in zip(epsilons, rewards_mean):  # Use epsilons and rewards_mean
    plt.plot(rewards, label=fr"$\epsilon = {epsilon:.2f}$")
plt.title("Figure 2.2: Average Reward Over Time")
plt.xlabel("Steps")
plt.ylabel("Average Reward")
plt.legend()

In [ ]:
plt.subplot(2, 1, 2)
for epsilon, counts in zip(epsilons, optimal_action_counts):
    plt.plot(counts, label=fr"$\epsilon = {epsilon:.2f}$")
plt.title("Figure 2.3: Optimal Actions Over Time")
plt.xlabel("Steps")
plt.ylabel("% Optimal action")
plt.legend()

In [ ]:
plt.savefig("../generated_images/figure_2_2.png")
plt.close()  # Close the plot to free up memory

# Print success message
print("Plot saved successfully at: ../generated_images/figure_2_2.png")

## 3. Optimistic Initial Values VS Realistic Initial Values

In [ ]:
# Create a list of 2 bandits where:
# 1. 1st bandit: ε = 0, 𝑄_1(𝑎) = 5, 𝛼 = 0.1,
# 2. 2nd bandit: ε = 0.1, 𝑄_1(𝑎) = 0, 𝛼 = 0.1
bandits = [Bandit(epsilon=0, initial_action_value_estimates=5, step_size=0.1),
           Bandit(epsilon=0.1, initial_action_value_estimates=0, step_size=0.1)]

In [ ]:
# Define number of runs
runs = 5000

# Define number of times
times = 10000

# Simulate optimal action counts
optimal_actions_mean, _ = simulate(runs, times, bandits)

In [ ]:
plt.plot(optimal_actions_mean[0], label=r"$\epsilon = 0, Q_1 = 5$")
plt.plot(optimal_actions_mean[1], label=r"$\epsilon = 0.1, Q_1 = 0$")

plt.title("Figure 2.3: Optimistic Initial Values VS Realistic Initial Values")
plt.xlabel("Steps")
plt.ylabel("% Optimal Action")
plt.legend()

In [ ]:
plt.savefig("../generated_images/figure_2_3.png")
plt.close()

# Print success message
print("Plot saved successfully at: ../generated_images/figure_2_3.png")

## 4. Upper-Confidence-Bound (UCB) Action Selection

In [3]:
# Create a list of 2 bandits where:
# 1. 1st bandit: ε = 0, 𝑐 = 2, uses sample-average method,
# 2. 2nd bandit: ε = 0.1, uses sample-average method
bandits = [Bandit(epsilon=0, confidence_level=2, use_sample_averages=True),
           Bandit(epsilon=0.1, use_sample_averages=True)]


In [4]:
# Define number of runs
runs = 2000

# Define number of times
times = 1000

# Simulate average rewards
_, avarage_rewards = simulate(runs, times, bandits)


100%|██████████| 2000/2000 [00:21<00:00, 90.94it/s]


In [6]:
# Plotting
plt.plot(avarage_rewards[0], label = 'UCB $c = 2$')
plt.plot(avarage_rewards[1], label = 'epsilon-greedy $epsilon =  0.1$')

plt.title("Figure 2.4: Average Reward Over Time")
plt.xlabel("Steps")
plt.ylabel("Average Reward")
plt.legend()

plt.savefig("../generated_images/figure_2_4.png")
plt.close()

## 5. Gradient Bandit Algorithms (GBA)

In [ ]:
# Create a list of 4 bandits where:
# 1. 1st bandit: uses GBA, 𝛼 = 0.1, uses average reward as baseline for GBA, expects true reward of 4,
# 2. 2nd bandit: uses GBA, 𝛼 = 0.1, doesn't use average reward as baseline for GBA, expects true reward of 4,
# 3. 3rd bandit: uses GBA, 𝛼 = 0.4, uses average reward as baseline for GBA, expects true reward of 4,
# 4. 4th bandit: uses GBA, 𝛼 = 0.4, doesn't use average reward as baseline for GBA, expects true reward of 4


In [ ]:
# Define number of runs


# Define number of times


# Simulate optimal action counts\


In [ ]:
# Labels


In [ ]:
# Plotting
